# **Store Data to Vector Store (OJK)**

Ini cara untuk storing ke Redis, tapi untuk [Load](#load) Document beda-beda untuk tiap data BI, OJK, dan SIKEPO. Jadi buat sendiri function `extract_all_documents_in_directory` nya

## **Setup**

In [15]:
# import nest_asyncio
# nest_asyncio.apply()

import warnings
warnings.filterwarnings("ignore")

## **Config**

In [16]:
from utils.config import get_config

config = get_config()

## **Define Model**

In [17]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_3_SMALL)

## **Indexing (All)**

In [4]:
def clean_document_content(content):
    return content.replace('\x00', '')  # Remove NUL characters

In [5]:
# from utils.documents_extractor.documents_extract_ojk import extract_all_documents_in_directory
# from utils.documents_split import document_splitter
# import pickle
# import os

# documents_dirs = ['./data/documents1/', './data/documents2/', './data/documents3/']
# pickle_path = './data/pickles/'
# metadata_path = './data/metadata/files_metadata.csv'

# LOAD_PICKLE = False

# for dir in documents_dirs:
#     if not LOAD_PICKLE:
#         documents = extract_all_documents_in_directory(dir, metadata_path, treshold=0.98)

#     if not LOAD_PICKLE:
#         all_splits = document_splitter(docs=documents)
#         all_splits_sorted = sorted(all_splits, key=lambda x: (x.metadata['doc_id'], x.metadata.get('page_number', '0')))
#         for split in all_splits_sorted:
#             split.page_content = clean_document_content(split.page_content)

#         # Determine the pickle file name based on the directory
#         dir_name = os.path.basename(os.path.normpath(dir))
#         file_pickle_name = f'{dir_name}.pkl'

#         # Save the sorted splits to a pickle file
#         with open(os.path.join(pickle_path, file_pickle_name), 'wb') as file:
#             pickle.dump(all_splits_sorted, file)

## **Indexing**

In [6]:
documents_dir = './data/documents1/'
pickle_path = './data/pickles/'
metadata_path = './data/metadata/files_metadata.csv'

LOAD_PICKLE = True

### **Load**

Untuk SIKEPO dan BI beda cara extract documentsnya, file document_extractor buat sendiri :D.

In [7]:
from utils.documents_extractor.documents_extract_ojk import extract_all_documents_in_directory

if not LOAD_PICKLE:
    documents = extract_all_documents_in_directory(documents_dir, metadata_path, treshold=0.98)

### **Split**

In [21]:
from utils.documents_split import document_splitter
import pickle

if not LOAD_PICKLE:
    all_splits = document_splitter(docs=documents)
    all_splits_sorted = sorted(all_splits, key=lambda x: (x.metadata['doc_id'], x.metadata.get('page_number', '0')))
    for split in all_splits_sorted:
        split.page_content = clean_document_content(split.page_content)
    # Open a file and use dump()
    with open(pickle_path + 'documents1.pkl', 'wb') as file:
        # A new file will be created
        pickle.dump(all_splits_sorted, file)

# Open the file in binary mode 
with open(pickle_path + 'documents3.pkl', 'rb') as file:
    # Call load method to deserialze 
    all_splits = pickle.load(file)

In [22]:
len(all_splits)

49201

In [23]:
all_splits[0].metadata

{'doc_id': 2212,
 'title': 'Surat Edaran Bank Indonesia Nomor 9/30/DPNP',
 'sector': None,
 'subsector': None,
 'regulation_type': None,
 'regulation_number': '9/30/DPNP/2007',
 'effective_date': '2007/12/12',
 'file_url': 'https://www.ojk.go.id/id/kanal/perbankan/regulasi/surat-edaran-bank-indonesia/Documents/187.pdf',
 'page_number': 1}

### **Storing**

In [24]:
from database.vector_store.vector_store import ElasticIndexManager

# vector_store_manager = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)
# vector_store_manager = PostgresIndexManager(index_name='ojk', embed_model=embed_model, config=config)
vector_store_manager = ElasticIndexManager(index_name='ojk', embed_model=embed_model, config=config)

# vector_store_manager.delete_index() # WARNING: This will delete the index
vector_store_manager.store_vector_index(docs=all_splits, batch_size=500)
vector_store = vector_store_manager.load_vector_index()

Start loading from idx: 0
Loaded 1-500 documents
Loaded 501-1000 documents
Loaded 1001-1500 documents
Loaded 1501-2000 documents
Loaded 2001-2500 documents
Loaded 2501-3000 documents
Loaded 3001-3500 documents
Loaded 3501-4000 documents
Loaded 4001-4500 documents
Loaded 4501-5000 documents
Loaded 5001-5500 documents
Loaded 5501-6000 documents
Loaded 6001-6500 documents
Loaded 6501-7000 documents
Loaded 7001-7500 documents
Loaded 7501-8000 documents
Loaded 8001-8500 documents
Loaded 8501-9000 documents
Loaded 9001-9500 documents
Loaded 9501-10000 documents
Loaded 10001-10500 documents
Loaded 10501-11000 documents
Loaded 11001-11500 documents
Loaded 11501-12000 documents
Loaded 12001-12500 documents
Loaded 12501-13000 documents
Loaded 13001-13500 documents
Loaded 13501-14000 documents
Loaded 14001-14500 documents
Loaded 14501-15000 documents
Loaded 15001-15500 documents
Loaded 15501-16000 documents
Loaded 16001-16500 documents
Loaded 16501-17000 documents
Loaded 17001-17500 documents
Loa

[2024-07-31 18:04:59,465] [ WARNING] _node_pool.py:249 - Node <Urllib3HttpNode(https://16.78.85.84:9200)> has failed for 1 times in a row, putting on 1 second timeout
[2024-07-31 18:04:59,466] [ WARNING] _transport.py:415 - Retrying request after failure (attempt 0 of 10)
Traceback (most recent call last):
  File "c:\Users\acer\miniconda3\envs\chatbot-ocbc\lib\site-packages\elastic_transport\_transport.py", line 342, in perform_request
    resp = node.perform_request(
  File "c:\Users\acer\miniconda3\envs\chatbot-ocbc\lib\site-packages\elastic_transport\_node\_http_urllib3.py", line 202, in perform_request
    raise err from None
elastic_transport.ConnectionError: Connection error caused by: ProtocolError(('Connection aborted.', timeout('The write operation timed out')))
[2024-07-31 18:05:07,211] [ WARNING] _node_pool.py:269 - Node <Urllib3HttpNode(https://16.78.85.84:9200)> has been marked alive after a successful request


Loaded 26501-27000 documents
Loaded 27001-27500 documents
Loaded 27501-28000 documents
Loaded 28001-28500 documents
Loaded 28501-29000 documents
Loaded 29001-29500 documents
Loaded 29501-30000 documents
Loaded 30001-30500 documents
Loaded 30501-31000 documents
Loaded 31001-31500 documents
Loaded 31501-32000 documents
Loaded 32001-32500 documents
Loaded 32501-33000 documents
Loaded 33001-33500 documents
Loaded 33501-34000 documents
Loaded 34001-34500 documents
Loaded 34501-35000 documents
Loaded 35001-35500 documents
Loaded 35501-36000 documents
Loaded 36001-36500 documents
Loaded 36501-37000 documents
Loaded 37001-37500 documents
Loaded 37501-38000 documents
Loaded 38001-38500 documents
Loaded 38501-39000 documents
Loaded 39001-39500 documents
Loaded 39501-40000 documents
Loaded 40001-40500 documents
Loaded 40501-41000 documents
Loaded 41001-41500 documents
Loaded 41501-42000 documents
Loaded 42001-42500 documents
Loaded 42501-43000 documents
Loaded 43001-43500 documents
Loaded 43501-4

## **NYOBA2**

In [11]:
vector_store.as_retriever().invoke("Berapa SWDKLLJ dari buldozer?")

[Document(metadata={'doc_id': 4, 'title': 'Tata Cara dan Mekanisme Penyampaian Data Transaksi Pendanaan dan Pelaporan Penyelenggara Layanan Pendanaan Bersama Berbasis Teknologi Informasi (LPBBTI)', 'sector': 'IKNB', 'subsector': 'Peraturan Lainnya', 'regulation_type': 'Surat Edaran OJK', 'regulation_number': '1/SEOJK.06/2024', 'effective_date': '2024/07/01', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/Tata-Cara-dan-Mekanisme-Penyampaian-Data-Transaksi-Pendanaan-dan-Pelaporan-Penyelenggara-Layanan-Pendanaan-Bersama-Berbasis/SEOJK%201-SEOJK.06-2024%20Tata%20Cara%20dan%20Mekanisme%20Penyampaian%20Data%20Transaksi%20Pendanaan%20dan%20Pelaporan%20Penyelenggara%20LPBBTI.pdf', 'page_number': 15}, page_content='No \nKode Jenis Kelamin \nPengisian \n1 \nBpk. Adiwena  \nLaki-Laki \n(sesuai dengan referensi) \n2 \nIbu Suci \nWanita \n(sesuai dengan referensi) \n3 \nPT ABC \nkolom dikosongkan \n \n15. Alamat \na. \nKolom ini diisi dengan alamat rumah untuk perorangan dan alamat 

In [12]:
from langchain_community.vectorstores.pgvector import PGVector
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)
from langchain_core.language_models.base import BaseLanguageModel
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.vectorstores import VectorStore
from langchain.retrievers.self_query.elasticsearch import ElasticsearchTranslator
from langchain_core.prompts import PromptTemplate
from langchain_core.structured_query import Operator, Comparator

# Define metadata field information
metadata_field_info = [
    AttributeInfo(
        name="title",
        description="The title of the document of regulation",
        type="string",
    ),
    AttributeInfo(
        name="sector",
        description="""The sector of the regulation""",
        type="string",
    ),
    AttributeInfo(
        name="subsector",
        description="The subsector of the regulation",
        type="string",
    ),
    AttributeInfo(
        name="regulation_type",
        description="""The type of the regulation""",
        type="string",
    ),
    AttributeInfo(
        name="regulation_number",
        description="The number of the regulation",
        type="string",
    ),
    AttributeInfo(
        name="effective_date",
        description="The effective date of the regulation in string format 'YYYY/MM/DD'",
        type="string",
    ),
]

# Define document content description
document_content_description = "The content of the document"

# Define prompt
SCHEMA = """\
<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{{{{
    "query": string \\ text string to compare to document contents
    "filter": string \\ logical condition statement for filtering documents
}}}}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` ({allowed_comparators}): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` ({allowed_operators}): logical operator
- `statement1`, `statement2`, ... (comparison statements or logical operation statements): one or more statements to apply the operation to

Make sure that you only use the comparators and logical operators listed above and no others.
Make sure that filters only refer to attributes that exist in the data source.
Make sure that filters only use the attributed names with its function names if there are functions applied on them.
Make sure that filters take into account the descriptions of attributes and only make comparisons that are feasible given the type of data being stored.
Make sure that filters are only used as needed. If there are no filters that should be applied return "NO_FILTER" for the filter value.
Make sure that date attributes are compared using ASCII comparison operators with the date in the string format "YYYY/MM/DD".
"""

SCHEMA_PROMPT = PromptTemplate.from_template(SCHEMA)

# prompt = get_query_constructor_prompt(
#     document_contents=document_content_description,
#     attribute_info=metadata_field_info,
#     schema_prompt=SCHEMA_PROMPT,
    
# )
# output_parser = StructuredQueryOutputParser.from_components()
# query_constructor = prompt | llm_model | output_parser

# query_constructor.invoke("Berikan dokumen yang berlaku pada tanggal 1 Januari 2023 hingga 1 Januari 2024")



# # Create query constructor
# def self_query_ojk(llm_model: BaseLanguageModel, vector_store: VectorStore, search_type: str = "similarity") -> SelfQueryRetriever:
#     retriever = SelfQueryRetriever.from_llm(
#         document_contents=document_content_description,
#         # enable_limit=False,
#         use_original_query=True,
#         llm=llm_model,
#         vectorstore=vector_store,
#         metadata_field_info=metadata_field_info,
#         structured_query_translator=PGVectorTranslator(),
#     )

#     return retriever



def self_query_ojk(llm_model: BaseLanguageModel, vector_store: VectorStore, search_type: str = "similarity") -> SelfQueryRetriever:
    prompt = get_query_constructor_prompt(
        document_contents=document_content_description,
        attribute_info=metadata_field_info,
        schema_prompt=SCHEMA_PROMPT,
    )
    output_parser = StructuredQueryOutputParser.from_components()
    query_constructor = prompt | llm_model | output_parser

    retriever = SelfQueryRetriever(
        query_constructor=query_constructor,
        vectorstore=vector_store,
        search_type=search_type,
        structured_query_translator=ElasticsearchTranslator(),
        verbose=True,
    )

    return retriever

retriever = self_query_ojk(llm_model=llm_model, vector_store=vector_store, search_type="similarity")

In [15]:
# 2024-07-01
from langchain.globals import set_debug

set_debug(True)

context = retriever.invoke('Berikan dokumen yang berlaku dari tahun 20023 hingga 2025')
context

[chain/start] [retriever:Retriever > chain:RunnableSequence] Entering Chain run with input:
{
  "query": "Berikan dokumen yang berlaku dari tahun 20023 hingga 2025"
}
[chain/start] [retriever:Retriever > chain:RunnableSequence > prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "Berikan dokumen yang berlaku dari tahun 20023 hingga 2025"
}
[chain/end] [retriever:Retriever > chain:RunnableSequence > prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [retriever:Retriever > chain:RunnableSequence > llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Your goal is to structure the user's query to match the request schema provided below.\n\n<< Structured Request Schema >>\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n```json\n{\n    \"query\": string \\ text string to compare to document contents\n    \"filter\": string \\ logical condition statement 

[Document(metadata={'doc_id': 4, 'title': 'Tata Cara dan Mekanisme Penyampaian Data Transaksi Pendanaan dan Pelaporan Penyelenggara Layanan Pendanaan Bersama Berbasis Teknologi Informasi (LPBBTI)', 'sector': 'IKNB', 'subsector': 'Peraturan Lainnya', 'regulation_type': 'Surat Edaran OJK', 'regulation_number': '1/SEOJK.06/2024', 'effective_date': '2024/07/01', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/Tata-Cara-dan-Mekanisme-Penyampaian-Data-Transaksi-Pendanaan-dan-Pelaporan-Penyelenggara-Layanan-Pendanaan-Bersama-Berbasis/SEOJK%201-SEOJK.06-2024%20Tata%20Cara%20dan%20Mekanisme%20Penyampaian%20Data%20Transaksi%20Pendanaan%20dan%20Pelaporan%20Penyelenggara%20LPBBTI.pdf', 'page_number': 15}, page_content='No \nKode Jenis Kelamin \nPengisian \n1 \nBpk. Adiwena  \nLaki-Laki \n(sesuai dengan referensi) \n2 \nIbu Suci \nWanita \n(sesuai dengan referensi) \n3 \nPT ABC \nkolom dikosongkan \n \n15. Alamat \na. \nKolom ini diisi dengan alamat rumah untuk perorangan dan alamat 